<a href="https://colab.research.google.com/github/phonhay103/anything/blob/main/MobileNet_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf

In [23]:
def depth(v, divisor=8, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v

def correct_pad(inputs, kernel_size):
    input_size = inputs.shape[1:3]
    if input_size[0] is None:
        adjust = (1, 1)
    else:
        adjust = (1 - input_size[0] % 2, 1 - input_size[1] % 2)
    correct = (kernel_size // 2, kernel_size // 2)
    return ((correct[0] - adjust[0], correct[0]),
            (correct[1] - adjust[1], correct[1]))
    
def relu(x):
    return tf.keras.layers.ReLU()(x)

def hard_sigmoid(x):
    return tf.keras.layers.ReLU(6.)(x+3.) * (1./6)

def hard_swish(x):
    return tf.keras.layers.Multiply()([hard_sigmoid(x), x])

In [27]:
def se_block(inputs, filters, se_ratio, prefix):
    x = tf.keras.layers.GlobalAveragePooling2D(name=prefix + 'squeeze_excite/AvgPool')(inputs)
    x = tf.keras.layers.Reshape((1, 1, filters))(x)
    x = tf.keras.layers.Conv2D(depth(filters * se_ratio), kernel_size=1, padding='same', name=prefix + 'squeeze_excite/Conv')(x)
    x = tf.keras.layers.ReLU(name=prefix + 'squeeze_excite/Relu')(x)
    x = tf.keras.layers.Conv2D(filters, kernel_size=1, padding='same', name=prefix + 'squeeze_excite/Conv_1')(x)
    x = hard_sigmoid(x)
    x = tf.keras.layers.Multiply(name=prefix + 'squeeze_excite/Mul')([inputs, x])
    return x

def inverted_res_block(x, expansion, filters, kernel_size, stride, se_ratio, activation, block_id):
    shortcut = x
    prefix = 'expanded_conv/'
    infilters = x.shape[-1]

    # Expand
    if block_id:
        prefix = 'expanded_conv_{}/'.format(block_id)
        x = tf.keras.layers.Conv2D(depth(infilters * expansion), kernel_size=1, padding='same', use_bias=False, name=prefix + 'expand')(x)
        x = tf.keras.layers.BatchNormalization(epsilon=1e-3, momentum=0.999, name=prefix + 'expand/BatchNorm')(x)
        x = activation(x)

    # Depthwise
    if stride == 2:
        x = tf.keras.layers.ZeroPadding2D(padding=correct_pad(x, 3), name=prefix + 'depthwise/pad')(x)
    x = tf.keras.layers.DepthwiseConv2D(kernel_size, strides=stride, padding='same' if stride == 1 else 'valid', use_bias=False, name=prefix + 'depthwise')(x)
    x = tf.keras.layers.BatchNormalization(epsilon=1e-3, momentum=0.999, name=prefix + 'depthwise/BatchNorm')(x)
    x = activation(x)

    # Squeeze & Excite
    if se_ratio:
        x = se_block(x, depth(infilters * expansion), se_ratio, prefix)

    # Project
    x = tf.keras.layers.Conv2D(filters, kernel_size=1, padding='same', use_bias=False, name=prefix + 'project')(x)
    x = tf.keras.layers.BatchNormalization(epsilon=1e-3, momentum=0.999, name=prefix + 'project/BatchNorm')(x)

    if stride == 1 and infilters == filters:
        x = tf.keras.layers.Add(name=prefix + 'Add')([shortcut, x])
    return x

In [28]:
def MobileNetV3Small(input_shape=None, alpha=1.0, minimalistic=False, last_point_ch=1024, dropout_rate=0.2, classes=1000, classifier_activation='softmax'):
    def _depth(d):
        return depth(d * alpha)

    # Minimalistic
    if minimalistic:
        kernel = 3
        activation = relu
        se_ratio = None
    else:
        kernel = 5
        activation = hard_swish
        se_ratio = 0.25
    
    # Input Shape
    if input_shape is None:
        input_shape = (None, None, 3)
    
    # Feature Extraction
    img_input = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.experimental.preprocessing.Rescaling(scale=1. / 127.5, offset=-1.)(img_input)
    x = tf.keras.layers.Conv2D(16, kernel_size=3, strides=(2, 2), padding='same', use_bias=False, name='Conv')(x)
    x = tf.keras.layers.BatchNormalization(epsilon=1e-3, momentum=0.999, name='Conv/BatchNorm')(x)
    x = activation(x)

    x = inverted_res_block(x, 1, _depth(16), 3, 2, se_ratio, relu, 0)
    x = inverted_res_block(x, 72. / 16, _depth(24), 3, 2, None, relu, 1)
    x = inverted_res_block(x, 88. / 24, _depth(24), 3, 1, None, relu, 2)
    x = inverted_res_block(x, 4, _depth(40), kernel, 2, se_ratio, activation, 3)
    x = inverted_res_block(x, 6, _depth(40), kernel, 1, se_ratio, activation, 4)
    x = inverted_res_block(x, 6, _depth(40), kernel, 1, se_ratio, activation, 5)
    x = inverted_res_block(x, 3, _depth(48), kernel, 1, se_ratio, activation, 6)
    x = inverted_res_block(x, 3, _depth(48), kernel, 1, se_ratio, activation, 7)
    x = inverted_res_block(x, 6, _depth(96), kernel, 2, se_ratio, activation, 8)
    x = inverted_res_block(x, 6, _depth(96), kernel, 1, se_ratio, activation, 9)
    x = inverted_res_block(x, 6, _depth(96), kernel, 1, se_ratio, activation, 10)

    last_conv_ch = depth(x.shape[-1] * 6)
    if alpha > 1.0:
        last_point_ch = depth(last_point_ch * alpha)
    
    x = tf.keras.layers.Conv2D(last_conv_ch, kernel_size=1, padding='same', use_bias=False, name='Conv_1')(x)
    x = tf.keras.layers.BatchNormalization(epsilon=1e-3, momentum=0.999, name='Conv_1/BatchNorm')(x)
    x = activation(x)
    x = tf.keras.layers.Conv2D(last_point_ch, kernel_size=1, padding='same', name='Conv_2')(x)
    x = activation(x)

    # Classification
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Reshape((1, 1, last_point_ch))(x)
    if dropout_rate > 0:
        x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.Conv2D(classes, kernel_size=1, padding='same', name='Logits')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Activation(activation=classifier_activation, name='Predictions')(x)
    
    # Model
    return tf.keras.Model(inputs=img_input, outputs=x)

In [38]:
def MobileNetV3Large(input_shape=None, alpha=1.0, minimalistic=False, last_point_ch=1280, dropout_rate=0.2, classes=1000, classifier_activation='softmax'):
    def _depth(d):
        return depth(d * alpha)

    # Minimalistic
    if minimalistic:
        kernel = 3
        activation = relu
        se_ratio = None
    else:
        kernel = 5
        activation = hard_swish
        se_ratio = 0.25
    
    # Input Shape
    if input_shape is None:
        input_shape = (None, None, 3)
    
    # Feature Extraction
    img_input = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.experimental.preprocessing.Rescaling(scale=1. / 127.5, offset=-1.)(img_input)
    x = tf.keras.layers.Conv2D(16, kernel_size=3, strides=(2, 2), padding='same', use_bias=False, name='Conv')(x)
    x = tf.keras.layers.BatchNormalization(epsilon=1e-3, momentum=0.999, name='Conv/BatchNorm')(x)
    x = activation(x)

    x = inverted_res_block(x, 1, _depth(16), 3, 1, None, relu, 0)
    x = inverted_res_block(x, 4, _depth(24), 3, 2, None, relu, 1)
    x = inverted_res_block(x, 3, _depth(24), 3, 1, None, relu, 2)
    x = inverted_res_block(x, 3, _depth(40), kernel, 2, se_ratio, relu, 3)
    x = inverted_res_block(x, 3, _depth(40), kernel, 1, se_ratio, relu, 4)
    x = inverted_res_block(x, 3, _depth(40), kernel, 1, se_ratio, relu, 5)
    x = inverted_res_block(x, 6, _depth(80), 3, 2, None, activation, 6)
    x = inverted_res_block(x, 2.5, _depth(80), 3, 1, None, activation, 7)
    x = inverted_res_block(x, 2.3, _depth(80), 3, 1, None, activation, 8)
    x = inverted_res_block(x, 2.3, _depth(80), 3, 1, None, activation, 9)
    x = inverted_res_block(x, 6, _depth(112), 3, 1, se_ratio, activation, 10)
    x = inverted_res_block(x, 6, _depth(112), 3, 1, se_ratio, activation, 11)
    x = inverted_res_block(x, 6, _depth(160), kernel, 2, se_ratio, activation, 12)
    x = inverted_res_block(x, 6, _depth(160), kernel, 1, se_ratio, activation, 13)
    x = inverted_res_block(x, 6, _depth(160), kernel, 1, se_ratio, activation, 14)

    last_conv_ch = depth(x.shape[-1] * 6)
    if alpha > 1.0:
        last_point_ch = depth(last_point_ch * alpha)
    
    x = tf.keras.layers.Conv2D(last_conv_ch, kernel_size=1, padding='same', use_bias=False, name='Conv_1')(x)
    x = tf.keras.layers.BatchNormalization(epsilon=1e-3, momentum=0.999, name='Conv_1/BatchNorm')(x)
    x = activation(x)
    x = tf.keras.layers.Conv2D(last_point_ch, kernel_size=1, padding='same', name='Conv_2')(x)
    x = activation(x)

    # Classification
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Reshape((1, 1, last_point_ch))(x)
    if dropout_rate > 0:
        x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.Conv2D(classes, kernel_size=1, padding='same', name='Logits')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Activation(activation=classifier_activation, name='Predictions')(x)
    
    # Model
    return tf.keras.Model(inputs=img_input, outputs=x)

In [35]:
model = MobileNetV3Small()
# model = tf.keras.applications.MobileNetV3Small()

In [36]:
model.summary()

Model: "MobilenetV3small"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
rescaling_5 (Rescaling)         (None, None, None, 3 0           input_9[0][0]                    
__________________________________________________________________________________________________
Conv (Conv2D)                   (None, None, None, 1 432         rescaling_5[0][0]                
__________________________________________________________________________________________________
Conv/BatchNorm (BatchNormalizat (None, None, None, 1 64          Conv[0][0]                       
___________________________________________________________________________________

In [37]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['acc'])